In [157]:
import sys
from functools import reduce
from itertools import product
from tqdm import tqdm
from pyvis.network import Network
import networkx as nx

In [171]:
def isproperpart(first, second):
    return first.issubset(second) and (first != second)

def fuse(pair):
    return pair[0] | pair[1]

def minimize(stateset):
    listset = sorted(stateset, key = len, reverse = True)
    n = 0
    while n < len(listset):
        for test in listset:
            if isproperpart(test,listset[n]):
                listset.pop(n)
                n = n-1
                continue
        n = n+1
    return frozenset(listset)
def propsfromlist(props):
    return [propfromstring(prop) for prop in props]

def propfromstring(string):
    components = string.split('|')
    return tuple([unipropfromstring(prop) for prop in components])

def unipropfromstring(string):
    bits = string.split(';')
    return (statesetfromstring(bits[0]),statefromstring(bits[1]))

def statefromstring(string):
    return frozenset(string)

def statesetfromstring(string):
    states = string.split(' ')
    return minimize([statefromstring(state) for state in states])

def posetfromlist(list, test, wrapper = lambda x: x):
    poset = nx.DiGraph()
    poset.add_nodes_from([item for item in list])
    for x in list: 
        for y in list:
            if x!=y and test(x,y):
                poset.add_edge(x,y)
    return poset

def bottomwedge(first, second):
    return minimize(map(fuse, product(first, second)))

def bottomvee(first, second):
    return minimize(first | second)     

def wedge(first, second):
    return (bottomwedge(first[0],second[0]),fuse([first[1],second[1]]))
    
def vee(first,second):
    return (bottomvee(first[0],second[0]),fuse([first[1],second[1]]))
        
def conjoin(first, second):
    return (wedge(first[0], second[0]), vee(first[1], second[1]))

def disjoin(first, second):
    return (vee(first[0], second[0]), wedge(first[1], second[1]))

def negate(x):
    return (x[1], x[0])

def negclose(c):
    n = 0
    while n < len(c):
        new = negate(c[n])
        if not (new in c):
            c.append(new) 
        n = n+1
    return c

def isconjunct(first,second):
    return conjoin(first,second) == second
    
def conjclose(c):
    n = 0
    while n < len(c):
        m = 0
        while m < n:
            new = conjoin(c[m], c[n])
            if not (new in c):
                c.append(new)
            m = m+1
        n = n+1
    return c

# Finally disjunction, which is just the same:

def disjclose(c):
    n = 0
    while n < len(c):
        m = 0
        while m < n:
            new = disjoin(c[m], c[n])
            if not (new in c):
                c.append(new)
            m = m+1
        n = n+1
    return c
def close(c):
    return(disjclose(conjclose(negclose(c))))

def to_dot( E ):
    res = [ 'digraph G { rankdir = BT; ' ]
    res.extend( [ '"{}"->"{}";'.format( *e ) for e in E ] )
    res.append( '}' )
    return '\n'.join( res )

In [178]:
atoms = propsfromlist(['r;r|g b;gb','g;g|r b;rb','b;b|r g;rg'])
propositions = close(atoms)
pairs = posetfromlist(propositions,isconjunct)
poset = nx.DiGraph()

for edge in pairs.edges:
    poset.add_edge(str(edge[0]),str(edge[1]))

In [179]:
g = Network(height = 800, width = 800,notebook = True)
g.toggle_hide_edges_on_drag(False)
g.barnes_hut()
g.from_nx(poset)
g.show("test.html")